In [28]:
import requests
import nltk
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
import pandas as pd

In [29]:
def scrapeWikipedia(url):
    try:
        response = requests.get(url)

        soup = BeautifulSoup(response.content, 'html.parser')
        body = soup.find(id='firstHeading').contents

        pattern = r'>(.*?)<'
        regex = re.compile(pattern)
        results = regex.findall(str(body), re.DOTALL)

        return results[0]
    except Exception as e:
        print(f'Error: {e}, URL: {url}')
        return False

In [59]:

response = requests.get(url='https://en.wikipedia.org/wiki/Machine_learning')

soup = BeautifulSoup(response.content, 'html.parser')
body = soup.find(id='bodyContent')

allLinks = soup.find(id='bodyContent').find_all('a')
onlyLinks = []
onlyTitle = []

dict_ = {'link','title'}
df = pd.DataFrame(dict_)

for link in allLinks:
    url = link.get('href')
    if not url or not url.startswith('/wiki/'):
        continue

    onlyLinks.append("https://en.wikipedia.org" + url)
onlyLinks = sorted(list(set(onlyLinks)))

for link in onlyLinks:
    onlyTitle.append(scrapeWikipedia(link))

article_text = soup.get_text()

Error: list index out of range, URL: https://en.wikipedia.org/wiki/ACM_Computing_Surveys
Error: list index out of range, URL: https://en.wikipedia.org/wiki/Apache_SystemML
Error: list index out of range, URL: https://en.wikipedia.org/wiki/ArXiv
Error: list index out of range, URL: https://en.wikipedia.org/wiki/ArXiv_(identifier)
Error: list index out of range, URL: https://en.wikipedia.org/wiki/Artificial_Intelligence:_A_Modern_Approach
Error: list index out of range, URL: https://en.wikipedia.org/wiki/File:AI_hierarchy.svg
Error: list index out of range, URL: https://en.wikipedia.org/wiki/File:Artificial_intelligence_prompt_completion_by_dalle_mini.jpg
Error: list index out of range, URL: https://en.wikipedia.org/wiki/File:CLIPS.jpg
Error: list index out of range, URL: https://en.wikipedia.org/wiki/File:Colored_neural_network.svg
Error: list index out of range, URL: https://en.wikipedia.org/wiki/File:Computer_Retro.svg
Error: list index out of range, URL: https://en.wikipedia.org/wiki

In [60]:
print(len(onlyLinks))
print(len(onlyTitle))

851
851


In [62]:
onlyLinks

['https://en.wikipedia.org/wiki/AAAI_Conference_on_Artificial_Intelligence',
 'https://en.wikipedia.org/wiki/ACM_Computing_Classification_System',
 'https://en.wikipedia.org/wiki/ACM_Computing_Surveys',
 'https://en.wikipedia.org/wiki/ADALINE',
 'https://en.wikipedia.org/wiki/AI_boom',
 'https://en.wikipedia.org/wiki/AI_control_problem',
 'https://en.wikipedia.org/wiki/AI_safety',
 'https://en.wikipedia.org/wiki/AI_takeover',
 'https://en.wikipedia.org/wiki/AI_winter',
 'https://en.wikipedia.org/wiki/AT%26T_Labs',
 'https://en.wikipedia.org/wiki/Action_selection',
 'https://en.wikipedia.org/wiki/Activation_function',
 'https://en.wikipedia.org/wiki/Active_learning_(machine_learning)',
 'https://en.wikipedia.org/wiki/Adaptive_website',
 'https://en.wikipedia.org/wiki/Adversarial_machine_learning',
 'https://en.wikipedia.org/wiki/Affective_computing',
 'https://en.wikipedia.org/wiki/Agriculture',
 'https://en.wikipedia.org/wiki/Alan_Mackworth',
 'https://en.wikipedia.org/wiki/Alan_Turing

In [51]:
processed_article = article_text.lower()
processed_article = re.sub(r'\s+', ' ', processed_article)
processed_article = re.sub(r'[!"#$%&\'()*+,-./:;<=>?@[\]^_`{|}~]','', processed_article)

In [52]:
df = pd.DataFrame({'link':onlyLinks, 'title': onlyTitle})

In [53]:
df = df[df['title'] != False]
df

,link,title


In [35]:

import openai
from config.config import OPENAI_API_KEY
from openai import OpenAI

client = OpenAI(api_key=OPENAI_API_KEY)

def get_embedding(text, model="text-embedding-ada-002"):
    text = text.replace("\n", " ")
    return client.embeddings.create(input = [text], model=model).data[0].embedding

df['ada_embedding'] = df.title.apply(lambda x: get_embedding(x, model='text-embedding-ada-002'))
# df.to_csv('embedded.csv', index=False)


In [36]:
df

,link,title,ada_embedding
1,/wiki/Statistical_learning_in_language_acquisi...,Statistical learning in language acquisition,"[-0.023959005251526833, 0.02220078743994236, 0..."
2,/wiki/Data_mining,Data mining,"[-0.03406612575054169, -0.008638196624815464, ..."
4,/wiki/Supervised_learning,Supervised learning,"[-0.011321621015667915, 0.0036667357198894024,..."
5,/wiki/Unsupervised_learning,Unsupervised learning,"[-0.018639473244547844, 0.00786251574754715, -..."
6,/wiki/Online_machine_learning,Online machine learning,"[-0.03978290408849716, -0.020232059061527252, ..."
...,...,...,...
1236,/wiki/Category:Articles_with_GND_identifiers,Category,"[0.009862851351499557, -0.0007628026651218534,..."
1237,/wiki/Category:Articles_with_J9U_identifiers,Category,"[0.009862851351499557, -0.0007628026651218534,..."
1238,/wiki/Category:Articles_with_LCCN_identifiers,Category,"[0.009882415644824505, -0.0007409197860397398,..."
1239,/wiki/Category:Articles_with_NDL_identifiers,Category,"[0.009862851351499557, -0.0007628026651218534,..."


In [37]:
from embeddings_utils import get_embedding, cosine_similarity
import os

def search_reviews(df, product_description, n=3, pprint=True):
   embedding = get_embedding(product_description, model='text-embedding-ada-002')
   df['similarities'] = df.ada_embedding.apply(lambda x: cosine_similarity(x, embedding))
   res = df.sort_values('similarities', ascending=False).head(n)
   return res

In [41]:
res = search_reviews(df, 'Machine Learning', n=-1)

In [42]:
res

,link,title,ada_embedding,similarities
651,/wiki/Automated_machine_learning,Automated machine learning,"[-0.04817207157611847, -0.007564541883766651, ...",0.950806
22,/wiki/Automated_machine_learning,Automated machine learning,"[-0.04817207157611847, -0.007564541883766651, ...",0.950806
6,/wiki/Online_machine_learning,Online machine learning,"[-0.03978290408849716, -0.020232059061527252, ...",0.933531
7,/wiki/Batch_learning,Online machine learning,"[-0.03978290408849716, -0.020232059061527252, ...",0.933531
509,/wiki/Machine_learning_control,Machine learning control,"[-0.03250792995095253, -0.009753751568496227, ...",0.924010
...,...,...,...,...
882,/wiki/ISSN_(identifier),ISSN,"[-0.009489736519753933, 0.0027516952250152826,...",0.747891
449,/wiki/Peptide_sequence,Protein primary structure,"[-0.018780061975121498, 0.008332273922860622, ...",0.747103
623,/wiki/KXEN_Inc.,KXEN Inc.,"[0.01803894154727459, -0.008644795045256615, 0...",0.746702
916,/wiki/Richard_O._Duda,Richard O. Duda,"[-0.006808327976614237, 0.002348219510167837, ...",0.741499
